In [ ]:
# | default_exp gsc_storage


In [ ]:
# | export
from sqlmodel import Session, select
from seo_rat.models import GSCAnalytics
from typing import List, Dict
from datetime import datetime
from seo_rat.gsc_client import GSCAuth
from seo_rat.gsc_storage import store_gsc_data
from seo_rat.gsc_storage import get_top_queries

In [ ]:
# | export
def store_gsc_data(session: Session, site_url: str, date: str, rows: List[Dict]):
    """Store GSC data with upsert (update or insert)"""
    for row in rows:
        # Check if record exists
        stmt = select(GSCAnalytics).where(
            GSCAnalytics.site_url == site_url,
            GSCAnalytics.date == date,
            GSCAnalytics.query == row["keys"][0] if len(row["keys"]) > 0 else None,
            GSCAnalytics.page == row["keys"][1] if len(row["keys"]) > 1 else None,
            GSCAnalytics.country == row["keys"][2] if len(row["keys"]) > 2 else None,
            GSCAnalytics.device == row["keys"][3] if len(row["keys"]) > 3 else None,
        )
        existing = session.exec(stmt).first()

        if existing:
            # Update
            existing.clicks = row["clicks"]
            existing.impressions = row["impressions"]
            existing.ctr = row["ctr"]
            existing.position = row["position"]
        else:
            # Insert
            analytics = GSCAnalytics(
                site_url=site_url,
                date=date,
                query=row["keys"][0] if len(row["keys"]) > 0 else None,
                page=row["keys"][1] if len(row["keys"]) > 1 else None,
                country=row["keys"][2] if len(row["keys"]) > 2 else None,
                device=row["keys"][3] if len(row["keys"]) > 3 else None,
                clicks=row["clicks"],
                impressions=row["impressions"],
                ctr=row["ctr"],
                position=row["position"],
            )
            session.add(analytics)

    session.commit()


In [ ]:
# | export
def get_top_queries(
    session: Session, site_url: str, start_date: str, end_date: str, limit: int = 10
) -> List[Dict]:
    """Get top performing queries"""
    from sqlalchemy import func

    stmt = (
        select(
            GSCAnalytics.query,
            func.sum(GSCAnalytics.clicks).label("total_clicks"),
            func.sum(GSCAnalytics.impressions).label("total_impressions"),
            func.avg(GSCAnalytics.position).label("avg_position"),
            func.avg(GSCAnalytics.ctr).label("avg_ctr"),
        )
        .where(
            GSCAnalytics.site_url == site_url,
            GSCAnalytics.date >= start_date,
            GSCAnalytics.date <= end_date,
            GSCAnalytics.query.isnot(None),
        )
        .group_by(GSCAnalytics.query)
        .order_by(func.sum(GSCAnalytics.clicks).desc())
        .limit(limit)
    )

    return [dict(row._mapping) for row in session.exec(stmt)]


In [ ]:
# | export
def get_page_analytics(
    session: Session, site_url: str, page_path: str, start_date: str, end_date: str
) -> Dict:
    """Get analytics for specific page"""
    from sqlalchemy import func

    stmt = select(GSCAnalytics).where(
        GSCAnalytics.site_url == site_url,
        GSCAnalytics.page.contains(page_path),
        GSCAnalytics.date >= start_date,
        GSCAnalytics.date <= end_date,
    )

    results = session.exec(stmt).all()

    return {
        "page_path": page_path,
        "total_clicks": sum(r.clicks for r in results),
        "total_impressions": sum(r.impressions for r in results),
        "avg_position": sum(r.position for r in results) / len(results)
        if results
        else 0,
        "avg_ctr": sum(r.ctr for r in results) / len(results) if results else 0,
        "top_queries": list(set(r.query for r in results if r.query))[:10],
    }


In [ ]:
# | export
def get_analytics_by_date_range(
    session: Session, site_url: str, start_date: str, end_date: str
) -> List[GSCAnalytics]:
    """Get all analytics for date range"""
    stmt = select(GSCAnalytics).where(
        GSCAnalytics.site_url == site_url,
        GSCAnalytics.date >= start_date,
        GSCAnalytics.date <= end_date,
    )
    return session.exec(stmt).all()


In [ ]:
# | export
def get_trends(
    session: Session,
    site_url: str,
    start_date: str,
    end_date: str,
    dimension: str = None,
) -> List[Dict]:
    """Get trends over time, optionally grouped by dimension"""
    from sqlalchemy import func

    if dimension:
        stmt = (
            select(
                GSCAnalytics.date,
                getattr(GSCAnalytics, dimension),
                func.sum(GSCAnalytics.clicks).label("clicks"),
                func.sum(GSCAnalytics.impressions).label("impressions"),
                func.avg(GSCAnalytics.position).label("avg_position"),
                func.avg(GSCAnalytics.ctr).label("avg_ctr"),
            )
            .where(
                GSCAnalytics.site_url == site_url,
                GSCAnalytics.date >= start_date,
                GSCAnalytics.date <= end_date,
            )
            .group_by(GSCAnalytics.date, getattr(GSCAnalytics, dimension))
            .order_by(GSCAnalytics.date)
        )
    else:
        stmt = (
            select(
                GSCAnalytics.date,
                func.sum(GSCAnalytics.clicks).label("clicks"),
                func.sum(GSCAnalytics.impressions).label("impressions"),
                func.avg(GSCAnalytics.position).label("avg_position"),
                func.avg(GSCAnalytics.ctr).label("avg_ctr"),
            )
            .where(
                GSCAnalytics.site_url == site_url,
                GSCAnalytics.date >= start_date,
                GSCAnalytics.date <= end_date,
            )
            .group_by(GSCAnalytics.date)
            .order_by(GSCAnalytics.date)
        )

    return [dict(row._mapping) for row in session.exec(stmt)]


In [ ]:
# | export
def get_analytics_by_country(
    session: Session, site_url: str, country: str, start_date: str, end_date: str
) -> List[Dict]:
    """Get analytics filtered by country"""
    from sqlalchemy import func

    stmt = (
        select(
            GSCAnalytics.query,
            func.sum(GSCAnalytics.clicks).label("clicks"),
            func.sum(GSCAnalytics.impressions).label("impressions"),
        )
        .where(
            GSCAnalytics.site_url == site_url,
            GSCAnalytics.country == country,
            GSCAnalytics.date >= start_date,
            GSCAnalytics.date <= end_date,
        )
        .group_by(GSCAnalytics.query)
    )

    return [dict(row._mapping) for row in session.exec(stmt)]


In [ ]:
# | export
def get_analytics_by_device(
    session: Session, site_url: str, device: str, start_date: str, end_date: str
) -> List[Dict]:
    """Get analytics filtered by device"""
    from sqlalchemy import func

    stmt = (
        select(
            GSCAnalytics.query,
            func.sum(GSCAnalytics.clicks).label("clicks"),
            func.sum(GSCAnalytics.impressions).label("impressions"),
        )
        .where(
            GSCAnalytics.site_url == site_url,
            GSCAnalytics.device == device,
            GSCAnalytics.date >= start_date,
            GSCAnalytics.date <= end_date,
        )
        .group_by(GSCAnalytics.query)
    )

    return [dict(row._mapping) for row in session.exec(stmt)]


In [ ]:
# | export
def store_single_date(session: Session, auth: GSCAuth, site_url: str, date: str) -> int:
    """Store GSC data for a single date"""
    from seo_rat.gsc_client import fetch_gsc_data

    rows = fetch_gsc_data(auth, site_url, date, date)
    store_gsc_data(session, site_url, date, rows)
    return len(rows)


In [ ]:
# | export
def store_date_range(
    session: Session, auth: GSCAuth, site_url: str, start_date: str, end_date: str
) -> Dict:
    """Store GSC data for a date range"""
    from datetime import datetime, timedelta
    from seo_rat.gsc_client import fetch_gsc_data
    import time

    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")

    results = {"successful": [], "failed": [], "total_records": 0}
    current = start

    while current <= end:
        date_str = current.strftime("%Y-%m-%d")
        try:
            rows = fetch_gsc_data(auth, site_url, date_str, date_str)
            store_gsc_data(session, site_url, date_str, rows)
            results["successful"].append(date_str)
            results["total_records"] += len(rows)
        except Exception as e:
            results["failed"].append({"date": date_str, "error": str(e)})

        current += timedelta(days=1)
        time.sleep(1)  # Rate limit

    return results
